In [1]:
# importing libraties
import xml.etree.cElementTree as ET
import os
import re
import numpy as np
import pandas as pd
from tqdm import tqdm
import json
import random
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import subprocess
from transformers import T5ForConditionalGeneration, T5Tokenizer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import sys
import string

random.seed(0)

# base path where we have all the bin files for sux-eng
apertium_mt_path = '../'

# NMT paths
src_file_location = '../data/NMT_temp_data/src.txt'
tgt_file_location = '../data/NMT_temp_data/tgt.txt'
weight_location = '../../Data/_step_10000.pt'


import warnings
warnings.filterwarnings('ignore')
chencherry = SmoothingFunction()

/opt/homebrew/anaconda3/envs/gsoc/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# # loading t5 model for grammer error correction
# model = T5ForConditionalGeneration.from_pretrained("Unbabel/gec-t5_small")
# tokenizer = T5Tokenizer.from_pretrained('t5-small')

# tokenizer = AutoTokenizer.from_pretrained("SafiUllahShahid/EnGECmodel")
# model = AutoModelForSeq2SeqLM.from_pretrained("SafiUllahShahid/EnGECmodel")

In [3]:
# function to run grammer error correction
# def grammer_error_correction(eng_sentences):
#     corrected_sentence = []
#     for eng_sentence in tqdm(eng_sentences):
#         tokenized_sentence = tokenizer(eng_sentence, max_length=512, truncation=True, padding='max_length', return_tensors='pt')
#         gec_result = tokenizer.decode(
#         model.generate(
#             input_ids = tokenized_sentence.input_ids,
#             attention_mask = tokenized_sentence.attention_mask, 
#             max_length=512,
#             num_beams=5,
#             early_stopping=True,
#         )[0],
#         skip_special_tokens=True, 
#         clean_up_tokenization_spaces=True
#         )
#         corrected_sentence.append(gec_result)

#     return corrected_sentence

## Supporting functions

In [4]:
def txt_file_read(filename):
    lines=[]
    with open(filename, "r") as f:
        for line in f:
            line=line.strip()
            lines.append(line)
    return lines

In [5]:
def txt_file_save(filename, sentence_list):
    with open(filename, 'w') as filehandle:
        for listitem in sentence_list:
            filehandle.write('%s\n' % listitem)

In [6]:
# english translation replacer
# we need to replace this so that it can be compiled with lexd apertium analyzer
def eng_word_processing(engword):
    word = engword.replace('’','').replace('\'','')
    return word

In [7]:
# sumerian form replacer
# we need to replace this so that it can be compiled with lexd analyzer as well as for bilingual dictionary
def sux_word_processing(suxword):
    processed_word = ''
    for i,char in enumerate(suxword):
        if char in string.punctuation:
            processed_word+='\\'+char
        else:
            processed_word+=char
    # word = suxword.replace('{','\{').replace('}','\}').replace('[','\[').replace(']','\]').replace('(','\(').replace(')','\)').replace('|','\|')\
    #     .replace('<','').replace('>','').replace('@','').replace('.','').replace(':','-').replace('x','') # to escape special meaning of brackets and 
    return processed_word

In [8]:
# rule based machine translation system
# Note - there is no api call, here we are calling os.system('bash code') as there is no inbuilt python library for apertium 
def rule_machine_translation(sux_sentences):
    sux_RBMT = []
    for sux_sentence in sux_sentences:
        try:
            sux_sentence = sux_word_processing(sux_sentence)
            # sux_sentence = " ".join(sux_sentence.strip().split())
            
            ## Calling apertium rule based Engine
            apertium_translation_command = f'''echo {sux_sentence} | apertium -d {apertium_mt_path} sux-eng'''
            output = subprocess.check_output(apertium_translation_command, shell=True)
            output = output.decode('ascii').strip().replace('x','').replace('#','').replace('*',' ').split()
            output = " ".join(output)
            sux_RBMT.append(output)
        except:
            sux_RBMT.append('')

    return sux_RBMT

In [9]:
# Neural machine translation
# Here also there is no python api call, rather it is a bash call using os.system
def nn_machine_translation(sux_sentence):
    txt_file_save(src_file_location, sux_sentence)

    os.system(f'''onmt_translate -model {weight_location}  -src {src_file_location} -output {tgt_file_location}''')    
    
    sux_NNMT = txt_file_read(tgt_file_location)
    
    return sux_NNMT

In [10]:
# def eng_reference_preprocessing(eng_tranlation_reference):
#     eng_tranlation_reference = eng_tranlation_reference.replace('-',' ').lower()
#     eng_tranlation_reference = ''.join(i for i in s if i not in string.punctuation)
#     return eng_tranlation_reference

In [11]:
# main function to call and process conll files 
# translating sentences from a directory containg conll files
def process_conll_files(dir_path):
    eng_reference_sentneces = []
    sumerian_sentences = []
    file_names = []

    # extracting sumerian and english translation from the file
    file_name = os.listdir(dir_path)

    # reading data from conll files
    for file in tqdm(file_name):

        file_path = os.path.join(dir_path,file)
        file_data = txt_file_read(file_path)

        # extracting data from conll files 
        sux_sentence = ''
        eng_tranlation_reference = ''
        for row in file_data:
            if row.startswith('# tr.en'):
                eng_tranlation_reference = row.split('tr.en:')[1]
            row_line = row.split('\t')
            if row_line[0].isdigit() and 'XPOSTAG' not in row:
                sux_sentence+=row_line[1]+' '

        # basic cleaning of english reference sentence so we do not miss correct words match because of basic errors like (Su-zen same as Suzen and suzen same as suzen)
        eng_tranlation_reference = eng_word_processing(eng_tranlation_reference)

        # basic cleaning of sumerian sentence before passing to rulebased translation
        # sux_sentence = sux_sentence.replace('<','').replace('>','').lower()


        eng_reference_sentneces.append(eng_tranlation_reference)
        sumerian_sentences.append(sux_sentence)
        file_names.append(file)


    return file_names, eng_reference_sentneces, sumerian_sentences

# TRANSLATION

## 1. Translating conll files

In [12]:
dir_path = '../data/mtaac_syntax_corpus_consolidated/dev/'

In [13]:
# extract file names and english, sumerian sentences
file_names, eng_tranlation_references, sux_sentences = process_conll_files(dir_path)

100%|██████████| 157/157 [00:00<00:00, 7849.17it/s]


#### a. Rule based machine translation 

In [14]:
sux_RBMT = rule_machine_translation(sux_sentences)

libc++abi: terminating with uncaught exception of type std::invalid_argument: unable to parse int


In [15]:
# grammer error correction if needed 
# gec_sux_RBMT = grammer_error_correction(sux_RBMT)

#### b. Neural machine translation 

In [16]:
sux_NNMT = nn_machine_translation(sux_sentences)

[2022-09-17 12:46:22,422 INFO] Translating shard 0.
/opt/homebrew/anaconda3/envs/gsoc/lib/python3.10/site-packages/onmt/translate/beam_search.py:209: UserWarning: An output with one or more elements was resized since it had shape [150], which does not match the required output shape [30, 5]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at  /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1659484612588/work/aten/src/ATen/native/Resize.cpp:24.)
  torch.mul(self.topk_scores, length_penalty, out=self.topk_log_probs)
/opt/homebrew/anaconda3/envs/gsoc/lib/python3.10/site-packages/onmt/translate/beam_search.py:212: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This resu

#### c. Calculating Bleu score

In [17]:
#================================ BLEU SCORE ================================================ #
translated_df = []
rule_bleu_list = []
nn_bleu_list = []


for file, eng_tranlation_reference, sux_sentence,sux_RB,sux_NN in tqdm(zip(file_names,eng_tranlation_references,sux_sentences,sux_RBMT,sux_NNMT)):

    rule_bleu = sentence_bleu([eng_tranlation_reference.split()], sux_RB.split(),smoothing_function=chencherry.method1,weights = (0.75,0.25,0,0))*100
    rule_bleu_list.append(rule_bleu)


    nn_bleu = sentence_bleu([eng_tranlation_reference.split()], sux_NN.split(), smoothing_function=chencherry.method1, weights = (0.75,0.25,0,0))*100
    nn_bleu_list.append(nn_bleu)

    translated_df.append([file, sux_sentence, eng_tranlation_reference, sux_RB, sux_NN, rule_bleu, nn_bleu])

    # if count==33:
    #     break


print(f'''\n average bleu score for rule based is''', (np.mean(rule_bleu_list),np.median(rule_bleu_list)))
print(f'''\n average bleu score for neural network based is''',(np.mean(nn_bleu_list),np.median(nn_bleu_list)))

157it [00:00, 3787.72it/s]


 average bleu score for rule based is (19.156694424387524, 20.451732834048002)

 average bleu score for neural network based is (18.86855003949193, 6.881713720035471)


In [18]:
# for test and analysis
col_name = ['file', 'sux_sentence', 'eng_tranlation_reference', 'eng_rule_based_translation', 'eng_nn_based_translation', 'rule_bleu', 'nn_bleu']
trainslation_pd = pd.DataFrame(translated_df,columns = col_name)
trainslation_pd.to_csv('../results/translation_mtaac_dev_results.csv')
trainslation_pd.head()

,file,sux_sentence,eng_tranlation_reference,eng_rule_based_translation,eng_nn_based_translation,rule_bleu,nn_bleu
0,P320163.conll,pisan-dub-ba mu 2(disz) sze-ba giri3-se3-ga ug...,Basket-of-tablets years of rations personnel ...,Basket-of-tablets one year ration attendant Ug...,Basketoftablets 2 years of the barley rations ...,11.908330,4.776235e+01
1,P125272.conll,usz2 ur-sila-luh 1(asz@c) GAN2 e2-ur2-bi-du10 ...,Dead Ur-silalu a-c- field E-urbidu a foreman ...,dead Ur-silaluh 1(towards his Na-silim@ c) fie...,the arasifieldwoods took in charge 1 acworker ...,17.613294,1.542663e-15
2,P416458.conll,4(disz) ki szu-{d}idim-ta mu-kux(DU) iti ezem-...,oxen cows male equids female equids old from ...,one from Szu-Idim delivery month in Ezemninazu...,4 garments from uIdim delivery of Ninazu year ...,28.661937,4.361240e+01
3,P107372.conll,pisan-dub-ba kiszib3 didli masz-x-x e2 lu2-gi-...,Basket-of-tablets sealed documents varied fro...,Basket-of-tablets several seal documents house...,Basketoftablets sealed documents varied from t...,17.467769,6.916163e+01
4,P101172.conll,5(disz) sila3 kasz saga 5(disz) sila3 ninda 5(...,sila fine beer sila bread shekels onions shek...,one unit good beer one unit one bread unit one...,for Inanna 5 sila3 fine beer 5 sila3 bread 5 s...,14.528770,5.314962e-03


## 2. Translating sumerian txt files

In [26]:
# txt file location
txt_file_location = '../test/sux-eng-input.txt'

In [27]:
sux_sentences_list = txt_file_read(txt_file_location)
sux_sentences_list

['pisan-dub-ba nig2-ka9-ak ab-ba-mu giri3 a-tu']

In [28]:
# a. rule based engine
# cat [file] | apertium -d . sux-eng
RBMT_result = rule_machine_translation(sux_sentences_list)
RBMT_result

['Basket-of-tablets account of Abbamu and under the authority of Atu']

In [29]:
# b. NMT engine
NMT_results = nn_machine_translation(sux_sentences_list)
NMT_results

[2022-09-17 20:04:20,859 INFO] Translating shard 0.
/opt/homebrew/anaconda3/envs/gsoc/lib/python3.10/site-packages/onmt/translate/beam_search.py:209: UserWarning: An output with one or more elements was resized since it had shape [5], which does not match the required output shape [1, 5]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at  /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1659484612588/work/aten/src/ATen/native/Resize.cpp:24.)
  torch.mul(self.topk_scores, length_penalty, out=self.topk_log_probs)
/opt/homebrew/anaconda3/envs/gsoc/lib/python3.10/site-packages/onmt/translate/beam_search.py:212: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results

['Basketoftablets accounts of Abbamu via Atu']

## 3. Translating single sumerian sentence

In [30]:
sux_sentence = "mu {d}szu-{d}suen lugal uri5{ki}-ma-ke4 ma2-dara3-abzu {d}en-ki-ka bi2-in-du8"

In [31]:
# a. rule based engine
RBMT_result = rule_machine_translation([sux_sentence])
# final_sentence = grammer_error_correction(RBMT_result)
RBMT_result

['year Szusuen king of Urim caulked Madaraabzu of Enkik']

In [32]:
# b. NMT engine
NNMT_results = nn_machine_translation([sux_sentence])
NNMT_results

[2022-09-17 20:09:42,692 INFO] Translating shard 0.
/opt/homebrew/anaconda3/envs/gsoc/lib/python3.10/site-packages/onmt/translate/beam_search.py:209: UserWarning: An output with one or more elements was resized since it had shape [5], which does not match the required output shape [1, 5]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at  /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1659484612588/work/aten/src/ATen/native/Resize.cpp:24.)
  torch.mul(self.topk_scores, length_penalty, out=self.topk_log_probs)
/opt/homebrew/anaconda3/envs/gsoc/lib/python3.10/site-packages/onmt/translate/beam_search.py:212: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results

['year By uSuen king of Ur the boat IbexofApsu of Enki was caulked']

In [35]:
# c. calculate bleu score 

# pass the reference senetnce 
eng_tranlation_reference = "in the year when Szusuen, king of Urim, caulked the Madaraabzu of Enki"
eng_tranlation_reference = eng_word_processing(eng_tranlation_reference)

rule_bleu = sentence_bleu([eng_tranlation_reference.split()], RBMT_result[0].split(),smoothing_function=chencherry.method1,weights = (0.75,0.25,0,0))*100
nn_bleu = sentence_bleu([eng_tranlation_reference.split()], NNMT_results[0].split(), smoothing_function=chencherry.method1, weights = (0.75,0.25,0,0))*100

print(f'''\n average bleu score for rule based is''', rule_bleu)
print(f'''\n average bleu score for neural network based is''',nn_bleu)


 average bleu score for rule based is 14.163412623658234

 average bleu score for neural network based is 16.900621985565852


# Exrta

### NLTK BlEU

In [ ]:

reference = [['this','boy']]
candidate = ['this', 'is']
chencherry = SmoothingFunction()
score = sentence_bleu(reference, candidate, smoothing_function=chencherry.method1, weights=(1,0,0,0))
print(score)

0.5


In [ ]:
sentence_bleu([eng_tranlation_reference.split()], eng_nn_based_translation.split(), smoothing_function=chencherry.method1)

### Huggingface evaluate

In [ ]:
import evaluate

/opt/homebrew/anaconda3/envs/gsoc/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
predictions = ["hello there"]
references = [
     ["hello bro"],
 ]
bleu = evaluate.load("sacrebleu")
results = bleu.compute(predictions=predictions, references=references, smooth_method= 'floor')
print(results)

{'score': 0.0, 'counts': [2, 1, 0, 0], 'totals': [2, 1, 0, 0], 'precisions': [100.0, 100.0, 0.0, 0.0], 'bp': 1.0, 'sys_len': 2, 'ref_len': 2}


In [ ]:
predictions = ["hello"]
references = [["hello there"]]
sacrebleu = evaluate.load("sacrebleu")
results = sacrebleu.compute(predictions=predictions, 
                             references=references, smooth_method = 'add-k')

In [ ]:
results

{'score': 36.78794411714425,
 'counts': [1, 1, 1, 1],
 'totals': [1, 1, 1, 1],
 'precisions': [100.0, 100.0, 100.0, 100.0],
 'bp': 0.36787944117144233,
 'sys_len': 1,
 'ref_len': 2}

## Torch check

In [2]:
import torch
import math

In [4]:
dtype = torch.float
device = torch.device("mps")

# Create random input and output data
x = torch.linspace(-math.pi, math.pi, 2000, device=device, dtype=dtype)
y = torch.sin(x)

# Randomly initialize weights
a = torch.randn((), device=device, dtype=dtype)
b = torch.randn((), device=device, dtype=dtype)
c = torch.randn((), device=device, dtype=dtype)
d = torch.randn((), device=device, dtype=dtype)

learning_rate = 1e-6
for t in range(2000):
    # Forward pass: compute predicted y
    y_pred = a + b * x + c * x ** 2 + d * x ** 3

    # Compute and print loss
    loss = (y_pred - y).pow(2).sum().item()
    if t % 100 == 99:
        print(t, loss)

# Backprop to compute gradients of a, b, c, d with respect to loss
    grad_y_pred = 2.0 * (y_pred - y)
    grad_a = grad_y_pred.sum()
    grad_b = (grad_y_pred * x).sum()
    grad_c = (grad_y_pred * x ** 2).sum()
    grad_d = (grad_y_pred * x ** 3).sum()

    # Update weights using gradient descent
    a -= learning_rate * grad_a
    b -= learning_rate * grad_b
    c -= learning_rate * grad_c
    d -= learning_rate * grad_d


print(f'Result: y = {a.item()} + {b.item()} x + {c.item()} x^2 + {d.item()} x^3')


99 3244.34814453125
199 2151.0498046875
299 1427.306884765625
399 948.1649780273438
499 630.9312744140625
599 420.8768310546875
699 281.77752685546875
799 189.65609741210938
899 128.6400909423828
999 88.22222900390625
1099 61.44561004638672
1199 43.70391082763672
1299 31.947072982788086
1399 24.154930114746094
1499 18.989810943603516
1599 15.565485000610352
1699 13.2947998046875
1799 11.788824081420898
1899 10.789817810058594
1999 10.126983642578125
Result: y = -0.009623829275369644 + 0.8226878643035889 x + 0.0016602700343355536 x^2 + -0.08848666399717331 x^3
